In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

## Rating making function (possibly fixed)

In [50]:
# Default weight on the factors
market_value_weight = 0.25 # Decide if we need this later
returns_weight = 0.5
error_weight = 0.25

# achieve data from yahoo finance
market_returns = 0.10 # Change later
total_market_value = 1000000000000 # Change later

start_date = '2021-01-01'
end_date = '2024-11-02'

# rating gives every stock a rating and sorts them in non-increasing order
# Four factors to be considered:
    # 1. Market Value (weight 25%)
    # 2. Returns (weight 50%)
    # 3. Error (weight 25%)
# The rating should between 0 and 1. A higher rating means the stock aligns better with the true market index.
def rating(dataframe):
    for stock, row in dataframe.iterrows():
        # Get the closing prices of the stock 
        stock_tick = yf.Ticker(row["stock"])
        stock_data = stock_tick.history(start=start_date, end=end_date, interval='1mo')[['Close']]
        stock_data.index = stock_data.index.strftime('%Y-%m-%d')
        # calculate returns for stock
        stock_returns_df = stock_data.ffill().pct_change().dropna()
        
        # Step 1: Calculate Market Value Score (Formula: Stock's Market Value / Total Market Value)
        stock_market_value = stock_tick.fast_info['marketCap']
        market_value_score = 1 - abs((stock_market_value - total_market_value) / total_market_value)
        dataframe.at[stock, "market value"] = market_value_score

        # Step 2: Calculate Returns Score (Formula: 1 - abs((Stock's Returns - Market Returns) / Market Returns))
        stock_returns = stock_returns_df['Close'].mean() 
        returns_score = 1 - abs((stock_returns - market_returns) / market_returns)
        dataframe.at[stock, "returns"] = returns_score

        # Step 3: Calculate tracking error score 
        tracking_error = (stock_returns_df['Close'] - (stock_returns_df["Close"]+market_returns)).std()
        dataframe.at[stock, "tracking error"] = tracking_error
        
    dataframe = dataframe.sort_values(by='tracking error', ascending=False)
    dataframe["Tracking error"] = 1 - abs((dataframe["tracking error"] - dataframe.at[0, 'tracking error']) / dataframe.at[0, 'tracking error'])

        # Step 4: Calculate rating of the stock based on weight
    dataframe["rating"] = dataframe["Tracking error"]*error_weight + dataframe["returns"]*returns_weight + dataframe["market value"]*market_value_weight

    # Sort the rating in descending order
    sorted_df = dataframe.sort_values(by='rating', ascending=False)
    return sorted_df

## Portfolio Build

In [56]:
port_build_date = "2024-11-18" # Change to the date we need later
port_build_end = "2024-11-19" # Has to be one day later

exchange_ticker = yf.Ticker('CADUSD=x')
exchange_rate_df = exchange_ticker.history(start=port_build_date, end=port_build_end, interval='1d') 
exchange_rate = exchange_rate_df["Close"].mean()
def build_portfolio(stocks):
   
    n = 1000000  # budget
    portfolio = [] 
    for stock, row in stocks.iterrows():
        
        # Grabbing Data
        stock = yf.Ticker(row["stock"])
        weight = row["weight"]
        stock_data = stock.history(start=port_build_date, end=port_build_end, interval='1d') 
        price = stock_data['Close'].mean() # price on the specified day
        
        
        # Exchanging currency if needed
        if stock.info["currency"] == "USD":
            price_cad = price * (1/exchange_rate)
        else: price_cad = price
            
        # Getting number of shares with fee
        shares_cost = n*weight
        shares = shares_cost / price_cad
        fee = max(3.95, shares * 0.001)
        fee_cost = shares_cost - fee
        shares = fee_cost / price_cad
        value = shares * price_cad
        
        # Adding our stock to portfolio
        portfolio.append({
            'Ticker': row["stock"],
            'Price': round(price_cad, 2),
            'Currency': stock.fast_info["currency"],
            'Shares': round(shares, 2), 
            'Value': round(value, 2),
            'Weight': row["weight"]  
        })
        
        # Note: Find a way to fix floating point rounding 
    
    # Creating our final dataframe  
    Portfolio_Final = pd.DataFrame(portfolio)
    Portfolio_Final.index += 1
    return Portfolio_Final



## Testing Stuff

In [3]:
stocks_test = pd.DataFrame({
    'stock': [
        'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'NVDA', 'META', 'NFLX', 'SHOP', 'JNJ', 
        'V', 'JPM', 'PG', 'XOM', 'UNH', 'MA', 'HD', 'CVX', 'PFE', 'KO', 'PEP', 'ABBV', 'TD.TO'
    ],
    'weight': [
        0.12, 0.11, 0.10, 0.08, 0.07, 0.06, 0.05, 0.04, 0.04, 0.04,
        0.03, 0.03, 0.03, 0.03, 0.03, 0.02, 0.02, 0.02, 0.02, 0.02,
        0.02, 0.01, 0.01
    ]
})
port_test = build_portfolio(stocks_test)
print(port_test)
print(port_test["Value"].sum())
print(port_test["Weight"].sum())

   Ticker    Price Currency  Shares      Value  Weight
1    AAPL   319.66      USD  375.39  119996.05    0.12
2    MSFT   582.85      USD  188.72  109996.05    0.11
3   GOOGL   245.75      USD  406.90   99996.05    0.10
4    AMZN   282.76      USD  282.91   79996.05    0.08
5    TSLA   474.88      USD  147.40   69996.05    0.07
6    NVDA   196.47      USD  305.36   59996.05    0.06
7    META   777.21      USD   64.33   49996.05    0.05
8    NFLX  1187.47      USD   33.68   39996.05    0.04
9    SHOP   148.43      USD  269.46   39996.05    0.04
10    JNJ   216.97      USD  184.34   39996.05    0.04
11      V   437.61      USD   68.54   29996.05    0.03
12    JPM   343.51      USD   87.32   29996.05    0.03
13     PG   239.37      USD  125.31   29996.05    0.03
14    XOM   168.66      USD  177.85   29996.05    0.03
15    UNH   826.62      USD   36.29   29996.05    0.03
16     MA   731.27      USD   27.34   19996.05    0.02
17     HD   575.39      USD   34.75   19996.05    0.02
18    CVX 

In [52]:
rating_test = rating(stocks_test)
rating_test.head()

,stock,weight,market value,returns,tracking error,Tracking error,rating
4,TSLA,0.07,0.912624,0.188291,1.403114e-17,1.0,0.572301
13,XOM,0.03,0.534314,0.281925,1.403114e-17,1.0,0.524541
6,META,0.05,0.600423,0.248410,1.403114e-17,1.0,0.524311
11,JPM,0.03,0.689843,0.191786,1.403114e-17,1.0,0.518354
10,V,0.03,0.601373,0.126570,1.403114e-17,1.0,0.463628
